<a href="https://colab.research.google.com/github/ChetHefton/4205_Group_Project/blob/main/loan_default_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#This cell imports necessary resources

import pandas as pd #data handling
import numpy as np #math and arrays

from sklearn.model_selection import train_test_split #split into train and validation
from sklearn.preprocessing import StandardScaler #normalizes
from sklearn.metrics import ( #import various metrics
    accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report
)

from sklearn.linear_model import LogisticRegression #baseline model
from sklearn.tree import DecisionTreeClassifier #use for comparison
#advance model 1 & 2 vvv
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt #visualize
import seaborn as sns #more visualize

In [8]:
#This cell loads the dataset

#download the dataset files directly from our git repo
!wget -O cs-training.csv https://raw.githubusercontent.com/ChetHefton/4205_Group_Project/main/cs-training.csv
!wget -O cs-test.csv https://raw.githubusercontent.com/ChetHefton/4205_Group_Project/main/cs-test.csv

#load into pandas
train = pd.read_csv("cs-training.csv")
test = pd.read_csv("cs-test.csv")

#show first 5 rows to confirm it's loaded correctly
train.head()



--2025-11-30 19:51:32--  https://raw.githubusercontent.com/ChetHefton/4205_Group_Project/main/cs-training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7564965 (7.2M) [text/plain]
Saving to: ‘cs-training.csv’

cs-training.csv     100%[===================>]   7.21M  --.-KB/s    in 0.1s    

2025-11-30 19:51:32 (61.4 MB/s) - ‘cs-training.csv’ saved [7564965/7564965]

--2025-11-30 19:51:32--  https://raw.githubusercontent.com/ChetHefton/4205_Group_Project/main/cs-test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4983329 

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [9]:
#this cell shows dataset info

#check shape of dataset
print("Training data shape:", train.shape)

#check for missing values
train.isnull().sum()

Training data shape: (150000, 12)


,0
Unnamed: 0,0
SeriousDlqin2yrs,0
RevolvingUtilizationOfUnsecuredLines,0
age,0
NumberOfTime30-59DaysPastDueNotWorse,0
DebtRatio,0
MonthlyIncome,29731
NumberOfOpenCreditLinesAndLoans,0
NumberOfTimes90DaysLate,0
NumberRealEstateLoansOrLines,0


In [10]:
#This cell shows basic statistics to identify outliers
train.describe()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,75000.500000,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,43301.414527,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37500.750000,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,75000.500000,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,112500.250000,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,150000.000000,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000
